In [ ]:
#hide
%load_ext autoreload
%autoreload 2

In [ ]:
#hide
from blobster import *

In [ ]:
# Cell
import json
import os
import glob
import io
import re
import zipfile
from datetime import date
from datetime import datetime
from functools import wraps
from pathlib import Path
import logging
import numpy as np
import pandas as pd
import pandas_profiling
import pandas_flavor as pf
import jdc

class Datapipe:
    def __init__(self, service=None, entity=None, meta=None, runtime=None):
        self.service: str = service
        self.runtime: str = runtime
        self.entity: str = entity
        self.meta = meta
        self.today = datetime.today().strftime("%Y-%m-%d")
        self.profile_report = None

datapipe = Datapipe()

In [ ]:
azure_blob_storage = AzureBlobStorage(account='vfudevblobstorage',
                                      key='eEbmjhexKPoGy+e8B2aUiDBPIG0qRThKRenJLgVbGN2TlcwMty9NopeUmwzIQ67DpYvcr8AwgPjobAiT0Zmzkw==')

In [ ]:
datapipe.service = 'vfu'
datapipe.runtime = 'local'

### items

In [ ]:
#vfu
@pf.register_dataframe_method
def step_consolidate_item_learning_levels(df, col, mapping=None):
    if mapping:
        learning_level_names = mapping
    else:
        learning_level_names = {'1': '1:Core', '2':'2:Intermediate', '3':'3:Advanced',
                    '4':'4:Expert', 'SCM Essentials':'1:Core', '5':'5:Others'}
    df[col] = df[col].str.split(' ').str[0]
    df[col] = np.where(df[col] == '', 'Missing Info', df[col])
    df[col] = df[col].map(learning_level_names)  
    df[col] = df[col].fillna(value='Missing Info')
    return df

/Users/tino/opt/anaconda3/envs/blobster/lib/python3.7/site-packages/pandas_flavor/register.py:31: UserWarning: registration of accessor <class 'pandas_flavor.register.register_dataframe_method.<locals>.inner.<locals>.AccessorMethod'> under name 'step_consolidate_item_learning_levels' for type <class 'pandas.core.frame.DataFrame'> is overriding a preexisting attribute with the same name.
  register_dataframe_accessor(method.__name__)(AccessorMethod)


In [ ]:
datapipe.entity = 'assets'

In [ ]:
#vfu
@pf.register_dataframe_method
def step_load_vfu_entity_definition(df, all_entities=False):
    """loads data definition of entity"""
    datapipe.definition = azure_blob_storage.blob_to_df(
        container_name=f"{datapipe.service}-data-definitions",
        blob_name=f"{datapipe.service}-data-definition.xlsx",
    )
    if not all_entities:
        datapipe.definition = datapipe.definition[
            datapipe.definition.entity == datapipe.entity
        ]
    return df

/Users/tino/opt/anaconda3/envs/blobster/lib/python3.7/site-packages/pandas_flavor/register.py:31: UserWarning: registration of accessor <class 'pandas_flavor.register.register_dataframe_method.<locals>.inner.<locals>.AccessorMethod'> under name 'step_load_vfu_entity_definition' for type <class 'pandas.core.frame.DataFrame'> is overriding a preexisting attribute with the same name.
  register_dataframe_accessor(method.__name__)(AccessorMethod)


In [ ]:
items_raw = pd.DataFrame().step_load_vfu_entity_definition()

NameError: name 'io' is not defined